In [5]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="darkgrid")

# CSV einlesen (Achtung keine '/' in den Spaltenüberschriften wegen späterem Namen)
# geht davon aus, dass keine Fehler in der Datei sind und alle Felder richtig gefüllt
# zum Überprüfen dien Notebooks LPS_first bzw. ARC_first nutzen

Team='ARC'
#Team='LPS'

if Team == 'ARC':
    anzahl_header_zeilen=0

    df = pd.read_csv('input/Ergebnisse Kanban Board PAM-EAv2.csv', sep=';', 
                     header=anzahl_header_zeilen, 
                     usecols=[2,3,5], 
                     parse_dates=[1,2], 
                     dayfirst=True,  #für deutsches Datumsformat
                     skip_blank_lines = False,
                     encoding='iso8859_15')

    # lies next als input (weil input nicht gepflegt)
    df.columns = ['art', 'input',  'out']
 
    print "Anzahl Zeilen eingelesen", len(df.index)    
    
    #Zeilen wegschmeissen, wenn mind. 2 Felder NaN sind
    df = df.dropna(thresh=2)
    print "Anzahl 'gültige' Zeilen ", len(df.index)

    print "Erste gültige Zeile\n", df[:1]
    print "\nLetzte gültige Zeile\n",df[-1:]

    print "\nDatatypes ", df.dtypes

    df = df[df.out.notnull()]
    print df
    #df[df.out.isnull()]
else:
    #df = pd.read_csv('input/20190620LPS_GebWartung.csv', sep=';', usecols=[9,11,18], parse_dates=[1,2], encoding='iso8859_15')
    df = pd.read_csv('input/LPS_20190806.csv', sep=';', usecols=[1,2,3], parse_dates=[1,2], encoding='iso8859_15')
    df.columns = ['art', 'input', 'out']
    
#end it TEAM
    
feiertage = ['2018-10-03', # Tag der Einheit
             '2018-11-01', # Allerheiligen
             '2018-12-24', # Heiligabend
             '2018-12-25', # Weihnachten
             '2018-12-26', # Weihnachten
             '2018-12-31', # Silvester
             '2019-01-01', # Neujahr
             '2019-02-28', # Wieverfastelovend
             '2019-03-04', # Rosenmontag
             '2019-04-19', # Karfreitag
             '2019-04-22', # Ostermontag
             '2019-05-01', # 1. Mai
             '2019-05-30', # Christi Himmelfahrt
             '2019-06-10', # Pfingst-Montag
             '2019-06-20'  # Fronleichnam
            ]

# astype('datetime64[D]') notwendig
dauer=np.busday_count(df.input.values.astype('datetime64[D]'), 
                  df.out.values.astype('datetime64[D]'), 
                    holidays=feiertage)+1

df['dauer'] = dauer

print df.dauer

von=(min(df.input))
von=von.strftime('%d.%m.%Y')

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')

def calc_8090_percentile(ldf, art, von, bis):
    perc80 = np.percentile(ldf.dauer,80,interpolation='lower') # lower ist richtig
    perc90 = np.percentile(ldf.dauer,90,interpolation='lower')
    return (perc80, perc90)
#end 8090_percentile
    

def plot_leadtime(ldf, art, von, bis):
    perc80 = np.percentile(ldf.dauer,80,interpolation='lower')
    perc90 = np.percentile(ldf.dauer,90,interpolation='lower')
    
    # gca stands for 'get current axis'
    ax = plt.gca()
    #hist zählt direkt die Elemente
    ldf.plot(kind='hist', y='dauer',bins=max(dauer),rwidth=1, color='black', ax=ax, label='')
    
    text80 = '80% fertig in ' + str(perc80) + ' Tagen'
    # vertical dotted line originating at mean value
    plt.axvline(perc80+.2, linestyle='-.', linewidth=1, color='blue', label=text80)

    text90 = '90% fertig in ' + str(perc90) + ' Tagen'
    # vertical dotted line originating at mean value
    plt.axvline(perc90+.2, linestyle='--', linewidth=1, color='red', label=text90)

    # Beschriftung
    plt.suptitle(art)
    
    title = von +' bis ' + bis + ': ' + str(len(ldf)) + ' Zettel'
    plt.title(title, fontsize=12)
    plt.xlabel('Dauer in Tagen')
    plt.ylabel('Anzahl')
    plt.legend(loc=0)
    
    # yticks ändern, so dass der Mindestabstand 1 ist
    ymin, ymax = plt.ylim()
    ax.yaxis.set_major_locator(plt.MultipleLocator(int(ymax/10+1)))    
    
    # x-Achse ab 0
    xmin, xmax = plt.xlim()
    plt.xlim(0, xmax)
    
    
    #plt.show()
    print Team
    print art
    filename='out/'+ Team + '_' + art +'.png'
    plt.savefig(filename, dpi=300)
    plt.close()
#enddef plotte()

von=(min(df.input))
von=von.strftime('%d.%m.%Y')
print von

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')
print bis

# muss kein DataFrame sein, ist meine komplizierte, aber laufende Lösung
arten = pd.DataFrame(columns=['Anzahl', 'Art'])
arten = arten.astype( dtype={'Anzahl': int, 'Art' : str})


for gew_art in df.art.drop_duplicates():
    gewaehlt = df.art == gew_art
#   gewaehlt = df.input > '2018-12-31'
    plot_leadtime(df[gewaehlt], gew_art, von, bis)
    laenge = len(df[gewaehlt])
    print gew_art, laenge
    arten.loc[len(arten)] = [int(laenge), gew_art]
#endfor gew_art in ....
print arten

plot_leadtime(df, 'Alle', von, bis)

Anzahl Zeilen eingelesen 92
Anzahl 'gültige' Zeilen  92
Erste gültige Zeile
           art      input        out
0  Regelarbeit 2019-05-20 2019-05-28

Letzte gültige Zeile
       art      input        out
91  Intern 2019-08-20 2019-08-22

Datatypes  art              object
input    datetime64[ns]
out      datetime64[ns]
dtype: object
               art      input        out
0      Regelarbeit 2019-05-20 2019-05-28
1      Regelarbeit 2019-04-16 2019-05-28
2      Regelarbeit 2019-05-15 2019-05-28
3           Intern 2019-05-07 2019-05-28
4           Intern 2019-04-02 2019-05-28
5           Intern 2019-05-14 2019-05-28
6         Fastlane 2019-05-14 2019-05-20
7           Intern 2019-04-16 2019-05-24
8           Intern 2019-05-13 2019-05-24
9           Intern 2019-04-16 2019-05-22
10          Intern 2019-04-30 2019-05-22
11          Intern 2019-04-18 2019-05-22
12          Intern 2019-05-14 2019-05-15
13     Regelarbeit 2019-03-05 2019-05-20
14          Intern 2019-05-07 2019-05-21
15     R

IndexError: cannot do a non-empty take from an empty axes.

In [4]:
def plot_cfd(ldf, art, von, bis):
    
    #alle Arbeitstage von ... bis
    bdate = pd.bdate_range(von, bis, holidays=feiertage, freq='C')
    Tage = bdate.strftime('%d.%m.%y')

    # erste Spalte des neuen Dataframes cfd
    cfd = pd.DataFrame(bdate, columns=['days'])
    cfd['Tage'] = Tage

    i = 0
    sum = 0
    col = np.zeros(len(cfd), int) 
    for day in cfd.days:
        sum += len(df[df.out == day])
        col[i] = sum
        i += 1
    #endfor day in cfd.days
    cfd['fertig'] = col

    i = 0
    sum = 0
    col = np.zeros(len(cfd), int) 
    for day in cfd.days:
        sum += len(df[df.input == day])
#         print str(day), len(df[df.input == day]), len(df[df.out == day])
        sum -= len(df[df.out == day])
        col[i] = sum
        if (col[i] < 0): 
            #             print "neg", str(day), str(col[i])
                col[i] = 0
                
        i += 1
    #endfor day in cfd.days
    
    cfd['Umsetzung'] = col
        
    ax=cfd.plot(kind='bar', stacked=True,  linewidth=.1 , x='Tage', y=[2,3])
    
    #nur jeden xstep-ten Wert plotten
    xstep = 10
    xindex = np.arange(2, len(cfd), xstep, int)
    
    einigeTage=Tage[xindex]
    ax.xaxis.set_major_locator(plt.MultipleLocator(xstep)) 
    ax.set_xticklabels(labels=einigeTage)
    
    filename='out/'+ Team + '_' + art +'.png'

    #ax.autoscale_view()
    plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right', size = 5) 
    
    # Beschriftung
    plt.suptitle('CFD von Team ' + Team)
   
    title = von +' bis ' + bis + ': ' + str(len(ldf)) + ' Zettel'
    plt.title(title)
    plt.ylabel('Anzahl')
    plt.xlabel('')
    
    plt.legend(loc=0)
    
    # noch Testen
    # x-Achse sinnvoll verschieben!! Jeweils manuel anzupassen!!!
    xmin, xmax = plt.xlim()
    plt.xlim(xmin+4, xmax-22)    
    
    plt.savefig(filename, dpi=300)
    plt.close()

#end plot_cfd

plot_cfd(df, 'CFD', von, bis)



In [ ]:
def plot_art(ldf, von, bis):
        

In [ ]:
# Breite des Intervals für den "gleitenden Durchschnitt" in Tagen
#breite = 

#alle Arbeitstage von ... bis
data = pd.bdate_range(von, bis, holidays=feiertage, freq='C')

# erste Spalte des neuen Dataframes perc
perc = pd.DataFrame(data, columns=['days'])

In [ ]:
i = 0
p80 = np.zeros(len(perc), int) 
for day in perc.days:
    gewaehlt = df.input > day
    if (len (df.dauer[gewaehlt]) != 0):
        p80[i]  = np.percentile(df.dauer[gewaehlt],90,interpolation='lower')
    i = i + 1
#endfor day in perc.days

perc['Alle'] = p80

for gew_art in df.art.drop_duplicates():
    i = 0
    p80 = np.zeros(len(perc), int) 
    gewaehlt = df.art == gew_art
    for day in perc.days:
        gewaehlt &= df.input > day
        if (len (df.dauer[gewaehlt]) != 0):
            p80[i]  = np.percentile(df.dauer[gewaehlt],90,interpolation='lower')
        i = i + 1
    #endfor day in perc.days
 #   perc[gew_art] = p80
#endfor gew_art in ...
perc.tail()

In [ ]:

perc.plot(x='days')

In [ ]:
perc.plot(x='days')
filename='out/'+ 'perc_over_time' +'.png'
plt.savefig(filename, dpi=300)
plt.close()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.pie(arten.Anzahl,  autopct='%1.1f%%', labels=arten.Art,
        startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()


In [ ]:
arten = arten.sort_values(by='Anzahl', ascending=False)

fig, ax = plt.subplots(figsize=(16, 8), subplot_kw=dict(aspect="equal"))

# Absolutwerte und Prozente
def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals)+.5)
    return "{:.0f}%\n({:d})".format(pct, absolute)


wedges, texts, autotexts = ax.pie(arten.Anzahl, autopct=lambda pct: func(pct, arten.Anzahl),
                                  textprops=dict(color="w"), counterclock=False, startangle = 150)

ax.legend(wedges, arten.Art,
          title="Arten der Arbeit",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.setp(autotexts, size=10, weight="bold")

ax.set_title(Team + ": Verteilung nach Arten der Arbeit")

#plt.show()
art='Arten_der_Arbeit'
filename='out/'+ Team + '_' + art +'.png'
plt.savefig(filename, dpi=300)
plt.close()